<a href="https://colab.research.google.com/github/josephthomaa/ML-Notebooks/blob/main/orca2_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Orca 2 Llama2 finetuned microsoft version

In [5]:
!pip install -q accelerate transformers

In [6]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install -q  llama-cpp-python --no-cache-dir

In [21]:
!pip install -q langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 6.6 MB/s eta 0:00:00


In [23]:
import torch
from huggingface_hub import hf_hub_download
from langchain.llms import LlamaCpp

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaForCausalLM,
    LlamaTokenizer,
)

In [24]:
model_path = hf_hub_download(
            repo_id="TheBloke/Orca-2-7B-GGUF",
            filename="orca-2-7b.Q4_K_M.gguf",
            resume_download=True,
            cache_dir="/.",
        )

orca-2-7b.Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

In [26]:
kwargs = {
            "model_path": model_path,
            "n_ctx": 4096,
            "max_tokens": 4096,
            "n_batch": 512,  # set this based on your GPU & CPU RAM
            "n_gpu_layers": 1
        }

llm = LlamaCpp(**kwargs)

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [29]:

llm("Ai is going to ")

In [35]:
import textwrap, time
from langchain import PromptTemplate, ConversationChain, LLMChain

prompt_template = """
Don't try to make up an answer, if you don't know just say that you don't know.
Answer in the same language the question was asked.
Use only the following pieces of context to answer the question at the end.

{context}

Question: {question}
Answer:"""


PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)


# retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

# qa_chain = RetrievalQA.from_chain_type(llm = llm,
#                                        chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
#                                        retriever = retriever,
#                                        chain_type_kwargs = {"prompt": PROMPT},
#                                        return_source_documents = True,
#                                        verbose = False)

In [32]:
def wrap_text_preserve_newlines(text, width=200): # 110
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])
    sources_used = ' \n'.join([str(source.metadata['source']) for source in llm_response['source_documents']])
    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [33]:
def llm_ans(query):
    start = time.time()
    llm_response = qa_chain(query)
    ans = process_llm_response(llm_response)
    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str